In [21]:
# Run this code once 
!pip install newsapi-python newspaper3l

/bin/sh: 1: pip3: not found


In [22]:
from newsapi import NewsApiClient
from newspaper import Article
import credentials
import query

In [35]:
def getArticle(url, return_text_only = True):
    article = Article(url)
    article.download()
    article.parse()
    if return_text_only:
        return article.text
    return article

In [36]:
getArticle(url)

'Fans set fire to flags inside the stadium\n\nGreek Super League side Panathinaikos say they "condemn unequivocally" the violence that saw their derby against city rivals Olympiakos abandoned.\n\nSunday\'s match was called off with 20 minutes to go after two pitch invasions and fighting between fans and police.\n\nPlayers on the visiting bench were attacked by Panathinaikos fans in the first pitch invasion after just four minutes at the Athens Olympic Stadium.\n\nOlympiakos had been leading 1-0 and were awarded the victory.\n\n"Panathinaikos FC condemns unequivocally every act of violence that took place at the Olympic Stadium and led to the permanent interruption of the derby," the club said.\n\nIt accused "a small minority of the 25,000 fans" of failing to respect football and added: "From the administration\'s point of view, absolute condemnation of all forms of athletic violence is essential."\n\nIt called on the Greek government to "finally show that it stands in the face of these

In [5]:
newsapi = NewsApiClient(credentials.retrieve_credential('news_api','aashish'))

In [9]:
per_page = 100

In [12]:
result = newsapi.get_everything(q = query.keywords[0], page_size=per_page,)

In [14]:
query.keywords[0]

'riot'

In [25]:
url = result['articles'][3]['url']

In [13]:
result

{'status': 'ok',
 'totalResults': 3602,
 'articles': [{'source': {'id': 'ars-technica', 'name': 'Ars Technica'},
   'author': 'Sam Machkovech',
   'title': 'Former Valve designer, writer dishes on his new “co-op” game studio',
   'description': 'Joined by ex-Riot dev: "Chet said one sentence. I said, \'Oh my god. That\'s the game.\'"',
   'url': 'https://arstechnica.com/gaming/2019/03/former-valve-designer-writer-dishes-on-his-new-co-op-game-studio/',
   'urlToImage': 'https://cdn.arstechnica.net/wp-content/uploads/2019/03/chet-and-kim-760x380.jpg',
   'publishedAt': '2019-03-18T15:28:58Z',
   'content': 'Enlarge/ Chet Faliszek, Dr. Kimberly Voll announce the creation of Stray Bombay, a new video game studio.\r\n1 with 1 posters participating\r\nSAN FRANCISCOIn 2017, game designer and writer Chet Faliszek left Valve Software. The departure was notable in part beca… [+6608 chars]'},
  {'source': {'id': 'the-new-york-times', 'name': 'The New York Times'},
   'author': 'JON PARELES, JON C